In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import random
import math
import copy
import gc
from tqdm import tqdm
from glob import glob

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore')

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141813 sha256=42cb3f77503a704636fd85caecacf154fea5d66a99a05f46f8cb0ca4da386351
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [8]:
! pip install konlpy
from konlpy.tag import Mecab

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 448 kB 31.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [12]:
np.set_printoptions(suppress = True)
temp_k = torch.tensor([[10,0,0],
                       [0, 10, 0],
                       [0, 0, 10],
                       [0, 0, 10]], dtype = torch.float32)
# 예시이지만 key는 임베딩 벡터 값

temp_v = torch.tensor([[10,0, 0],
                       [0,10, 0],
                       [0, 0, 10],
                       [0,0, 10]], dtype = torch.float32)
# 예시이지만 value는 임베딩 벡터 값 

temp_q = torch.tensor([[0,10,0]], dtype = torch.float32)



'\n- 여기서는 안써있지만 각 단어에 대해 임베딩 벡터값을 그대로 가져오는게 아님 \n- 각 단어를 표시하는 차원을 head개수로 나눠 변환하여 Query, Key, Value를 만들어냄 \n'

In [18]:
trans_k = torch.transpose(temp_k, -2, -1)
# query와 연관성이 있는 단어를 찾기위해 각 단어에 대해서 matmul을 수행하여 연관성 값 도출 
trans_k

tensor([[10.,  0.,  0.,  0.],
        [ 0., 10.,  0.,  0.],
        [ 0.,  0., 10., 10.]])

In [14]:
temp_k.size()

torch.Size([4, 3])

In [26]:
at_s = torch.matmul(temp_q, trans_k)
print(at_s)
scaled = at_s / math.sqrt(temp_k.size()[-1])
print(scaled)
at_w = torch.nn.functional.softmax(scaled, dim = -1)
print(at_w)
out = torch.matmul(at_w, temp_v)
print(out)
# 

tensor([[  0., 100.,   0.,   0.]])
tensor([[ 0.0000, 57.7350,  0.0000,  0.0000]])
tensor([[8.4333e-26, 1.0000e+00, 8.4333e-26, 8.4333e-26]])
tensor([[8.4333e-25, 1.0000e+01, 1.6867e-24]])


In [62]:
def scaled_dot_product_attention(q, k, v, mask):
  trans_k = torch.transpose(k, -2, -1)
  # query와 연관성이 있는 단어를 찾기위해 각 단어에 대해서 matmul을 수행하여 연관성 값 도출 

  matmul_qk = torch.matmul(q, trans_k)

  dk = k.size()[-1]
  # 각 단어의 벡터를 알기위해서 사용 

  scaled_attention_logits = matmul_qk / math.sqrt(dk)
  # scaling, 너무 큰값을 가지면 모델에 좋지않기 때문에 사용하는듯? 
  # 이게 attention score임 

  if mask is not None:
    scaled_attention_logits += (mask * -1e9)
    # mask가 있다면 mask tensor를 추가 

  attention_weights = torch.nn.functional.softmax(scaled_attention_logits , dim = -1)
  # attention score의 분포를 softmax함수를 통해서 확률로써 도출 
  # 이렇게되면 해당 query에 대해 key들의 영향을 확률로써 확인가능 

  output = torch.matmul(attention_weights, v)
  # 도출된 확률을 바탕으로 실제 value vector와 matmul
  # 근데 이렇게 곱하면 각 단어에 대한 value와 attention score를 곱하는게 아니지 않나?...

  return output, attention_weights
  

In [69]:
x = torch.rand(1, 60, 512)
# x = [batch_size, max_len(encoder), embedding_size]
m_h = MultiHeadAttention(d_model = 512, num_heads = 8)

In [82]:
s_c = x.transpose(1,2).contiguous().view(1, -1, num_heads * depth)
s_c.shape

torch.Size([1, 60, 512])

In [79]:
d_model, num_heads = 512, 8
depth = d_model // num_heads

# query 계산 과정 
wq = nn.Linear(d_model, d_model)
q = wq(x)
print(q.shape)
q = q.view(1, -1, 8, depth).transpose(1,2)
print(q.shape)
# 각 헤드에 들어가는 query vertor, value와 key도 똑같이 적용 




torch.Size([1, 60, 512])
torch.Size([1, 8, 60, 64])


In [70]:
out, at_w = m_h(v = x, k = x, q = x, mask = None)

In [73]:
print(out.shape, at_w.shape)


torch.Size([1, 60, 512]) torch.Size([1, 8, 60, 60])


In [67]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.model = d_model

    assert d_model % self.num_heads == 0 
    # 딱 나누어 떨어지도록 d_model과 num_heads를 잘설계해야함 
    # assert : 뒤의 조건이 TRUE가 아니면 error 발생 시킴 

    self.depth = d_model // self.num_heads
    
    self.wq = nn.Linear(d_model, d_model)
    self.wk = nn.Linear(d_model, d_model)
    self.wv = nn.Linear(d_model, d_model)

    self.wo = nn.Linear(d_model, d_model)

  def forward(self, v, k ,q, mask):
    batch_size = q.size()[0]
    # 배치사이즈에 맞에 query가 들어갈 것임으로 

    q = self.wq(q).view(batch_size, -1, self.num_heads, self.depth).transpose(1,2)
    # q : [batch_size, num_head, max_len, depth]
    # 단어 임베딩을 사용하는게 아니라 기존의 단어 임베딩 벡터로부터 헤더 개수를 나눈 것을 벡터의 차원으로 사용하여 단어를표현 
    
    k = self.wk(k).view(batch_size, -1, self.num_heads, self.depth).transpose(1,2)
    v = self.wv(v).view(batch_size, -1, self.num_heads, self.depth).transpose(1,2)
    # q와 똑같으므로 생략 

    scaled_attention, attention_weights = scaled_dot_product_attention(q,k,v,mask)

    scaled_attention = scaled_attention.transpose(1,2).contiguous().view(batch_size, -1, self.num_heads * self.depth)
    # 여기서 차원 헷갈림 ...
    # 여기서 최종적으로 sacled_attention [batch_size, max_len, embedding_size]로 차원을 맞춰줌 

    output = self.wo(scaled_attention)
    # 한번 더 선형회귀를 거쳐서 최종 output도출 

    return output, attention_weights
    # output : [batch_size, max_len, embedding_size]
    # attention_weights : [batch_size, num_heads, max_len, embedding_size]
    # attnetion_weights는 헤더의 개수마다 softmax값임으로 차원이 하나 늘어난것 같음 

'''
- 여기서는 안써있지만 각 단어에 대해 임베딩 벡터값을 그대로 가져오는게 아님 
- 각 단어를 표시하는 차원을 head개수로 나눠 변환하여 Query, Key, Value vector를 만들어냄 
''' 